# Installs

## wandb

You will need to fetch your api key from wandb.ai

In [1]:
# !pip install wandb -q

In [2]:
import wandb
wandb.login(key="e0408f5d7b96be3d00be30b39eda0f1e259672ed")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: gyuseoklee. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/gyuseok/.netrc


True

In [3]:
config = {}
config["epochs"] = 50
config["project"] = "Transform"
config["model_save"] = "/home/gyuseok/CMU/IDL/HW3/weight"
config["batch_size"] = 32

In [4]:
run = wandb.init(
    name = config["project"], ## Wandb creates random run names if you skip this field
    reinit = True, ### Allows reinitalizing runs when you re-run this cell
    # run_id = ### Insert specific run id here if you want to resume a previous run
    # resume = "must" ### You need this to resume previous runs, but comment out reinit = True when using this
    project = "hw3p2-ablations", ### Project should be created in your wandb account 
    config = config ### Wandb Config for your run
)

## Levenshtein

This may take a while

In [5]:
# !pip install python-Levenshtein
# !git clone --recursive https://github.com/parlance/ctcdecode.git
# !pip install wget
# %cd ctcdecode
# !pip install .
# %cd ..

# !pip install torchsummaryX

## imports

In [6]:
import torch
import random
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torchsummaryX import summary
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence

import torchaudio.transforms as tat

from sklearn.metrics import accuracy_score
import gc

import zipfile
import pandas as pd
from tqdm import tqdm
import os
import datetime

# imports for decoding and distance calculation
import ctcdecode
import Levenshtein
from ctcdecode import CTCBeamDecoder

import warnings
warnings.filterwarnings('ignore')

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print("Device: ", device)

Device:  cuda:0


# Kaggle Setup

In [7]:
# !pip install --upgrade --force-reinstall --no-deps kaggle==1.5.8
# #!mkdir /root/.kaggle

# with open("/home/gyuseok/.kaggle/kaggle.json", "w+") as f:
#     f.write('{"username":"leepro","key":"2842f0b3a1e5a14ccddb8ba7fc0a016b"}') 

# !chmod 600 /home/gyuseok/.kaggle/kaggle.json

In [8]:
# !kaggle competitions download -c 11-785-f22-hw3p2

In [9]:
# '''
# This will take a couple minutes, but you should see at least the following:
# 11-785-f22-hw3p2.zip  ctcdecode  hw3p2
# '''
# !unzip -q 11-785-f22-hw3p2.zip
# !ls

# Google Drive

In [10]:
# from google.colab import drive
# drive.mount('/content/gdrive')

# Dataset and Dataloader

In [11]:
# ARPABET PHONEME MAPPING
# DO NOT CHANGE
# This overwrites the phonetics.py file.

CMUdict_ARPAbet = {
    "" : " ", # BLANK TOKEN
    "[SIL]": "-", "NG": "G", "F" : "f", "M" : "m", "AE": "@", 
    "R"    : "r", "UW": "u", "N" : "n", "IY": "i", "AW": "W", 
    "V"    : "v", "UH": "U", "OW": "o", "AA": "a", "ER": "R", 
    "HH"   : "h", "Z" : "z", "K" : "k", "CH": "C", "W" : "w", 
    "EY"   : "e", "ZH": "Z", "T" : "t", "EH": "E", "Y" : "y", 
    "AH"   : "A", "B" : "b", "P" : "p", "TH": "T", "DH": "D", 
    "AO"   : "c", "G" : "g", "L" : "l", "JH": "j", "OY": "O", 
    "SH"   : "S", "D" : "d", "AY": "Y", "S" : "s", "IH": "I",
    "[SOS]": "[SOS]", "[EOS]": "[EOS]"}

CMUdict = list(CMUdict_ARPAbet.keys())
ARPAbet = list(CMUdict_ARPAbet.values())

PHONEMES = CMUdict
mapping = CMUdict_ARPAbet
LABELS = ARPAbet


### Train Data

In [12]:
import torchaudio

class AudioDataset(torch.utils.data.Dataset):

    # For this homework, we give you full flexibility to design your data set class.
    # Hint: The data from HW1 is very similar to this HW

    #TODO
    def __init__(self, origin_path, train = True): 
        '''
        Initializes the dataset.

        INPUTS: What inputs do you need here?
        '''
        self.train = train
        # Load the directory and all files in them
        
        self.origin_path = origin_path #"/home/gyuseok/CMU/HW3/hw3p2/train-clean-360"
        
        self.mfcc_dir = os.path.join(self.origin_path,"mfcc")
        self.transcript_dir = os.path.join(self.origin_path, "transcript","raw")

        self.mfcc_files = sorted(os.listdir(self.mfcc_dir)) #TODO
        self.transcript_files = sorted(os.listdir(self.transcript_dir)) #TODO

        self.PHONEMES = PHONEMES

        #TODO
        # WHAT SHOULD THE LENGTH OF THE DATASET BE?
        self.length = len(self.mfcc_files)
        
        #TODO
        # HOW CAN WE REPRESENT PHONEMES? CAN WE CREATE A MAPPING FOR THEM?
        # HINT: TENSORS CANNOT STORE NON-NUMERICAL VALUES OR STRINGS
        PHONEMES_dict = {letter:idx for idx,letter in enumerate(self.PHONEMES)}


        #TODO
        # CREATE AN ARRAY OF ALL FEATU ERS AND LABELS
        # WHAT NORMALIZATION TECHNIQUE DID YOU USE IN HW1? CAN WE USE IT HERE?
        self.mfccs = []
        self.transcripts = []
        
        for i in range(self.length):
            mfcc_path = os.path.join(self.mfcc_dir, self.mfcc_files[i])
            label_path = os.path.join(self.transcript_dir, self.transcript_files[i])
            
            mfcc = np.load(mfcc_path)
            label = np.load(label_path)
            label = np.vectorize(PHONEMES_dict.get)(label) # transform into number
            
            self.mfccs.append(mfcc)
            self.transcripts.append(label)
        
        '''
        You may decide to do this in __getitem__ if you wish.
        However, doing this here will make the __init__ function take the load of
        loading the data, and shift it away from training.
        '''
       

    def __len__(self):
        
        '''
        TODO: What do we return here?
        '''
        return self.length

    def __getitem__(self, ind):
        '''
        TODO: RETURN THE MFCC COEFFICIENTS AND ITS CORRESPONDING LABELS

        If you didn't do the loading and processing of the data in __init__,
        do that here.

        Once done, return a tuple of features and labels.
        '''
        mfcc = self.mfccs[ind] # TODO
        transcript = self.transcripts[ind] # TODO
        
        mfcc = torch.FloatTensor(mfcc)
        transcript = torch.LongTensor(transcript)
        return mfcc, transcript


    def collate_fn(self, batch):
        '''
        TODO:
        1.  Extract the features and labels from 'batch'
        2.  We will additionally need to pad both features and labels,
            look at pytorch's docs for pad_sequence
        3.  This is a good place to perform transforms, if you so wish. 
            Performing them on batches will speed the process up a bit.
        4.  Return batch of features, labels, lenghts of features, 
            and lengths of labels.
        '''
        # batch of input mfcc coefficients
        batch_mfcc = []
        batch_transcript = []
        lengths_mfcc = []
        lengths_transcript = []
        for x,y in batch:
            batch_mfcc.append(x) # TODO
            batch_transcript.append(y) # TODO
            lengths_mfcc.append(x.shape[0])
            lengths_transcript.append(y.shape[0])
            
        # HINT: CHECK OUT -> pad_sequence (imported above)
        # Also be sure to check the input format (batch_first)
        batch_mfcc_pad = pad_sequence(batch_mfcc, batch_first = True) # TODO
        batch_transcript_pad = pad_sequence(batch_transcript, batch_first = True) # TODO

        # You may apply some transformation, Time and Frequency masking, here in the collate function;
        # Food for thought -> Why are we applying the transformation here and not in the __getitem__?
        #                  -> Would we apply transformation on the validation set as well?
        #                  -> Is the order of axes / dimensions as expected for the transform functions?
        if self.train:
            T_mask = torchaudio.transforms.TimeMasking(time_mask_param = 7)
            F_mask = torchaudio.transforms.FrequencyMasking(freq_mask_param = batch_mfcc_pad.shape[1]//2)
            batch_mfcc_pad = F_mask(T_mask(batch_mfcc_pad))
        
        # Return the following values: padded features, padded labels, actual length of features, actual length of the labels
        return batch_mfcc_pad, batch_transcript_pad, torch.tensor(lengths_mfcc), torch.tensor(lengths_transcript)

       

### Test Data

In [13]:
# Test Dataloader
#TODO
class AudioDatasetTest(torch.utils.data.Dataset):
    def __init__(self, origin_path): 
        '''
        Initializes the dataset.

        INPUTS: What inputs do you need here?
        '''

        # Load the directory and all files in them
        self.origin_path = origin_path #"/home/gyuseok/CMU/HW3/hw3p2/train-clean-360"
        self.mfcc_dir = os.path.join(self.origin_path,"mfcc")
        self.mfcc_files = sorted(os.listdir(self.mfcc_dir)) #TODO


        #TODO
        # WHAT SHOULD THE LENGTH OF THE DATASET BE?
        self.length = len(self.mfcc_files)
   
        self.mfccs = []        
        for i in range(self.length):
            mfcc_path = os.path.join(self.mfcc_dir, self.mfcc_files[i])            
            mfcc = np.load(mfcc_path)
            self.mfccs.append(mfcc)
            
    def __len__(self):
        return self.length

    def __getitem__(self, ind):
        mfcc = self.mfccs[ind] # TODO
        mfcc = torch.FloatTensor(mfcc)
        return mfcc


    def collate_fn(self,batch):
        '''
        TODO:
        1.  Extract the features and labels from 'batch'
        2.  We will additionally need to pad both features and labels,
            look at pytorch's docs for pad_sequence
        3.  This is a good place to perform transforms, if you so wish. 
            Performing them on batches will speed the process up a bit.
        4.  Return batch of features, labels, lenghts of features, 
            and lengths of labels.
        '''
        # batch of input mfcc coefficients
        batch_mfcc = []
        lengths_mfcc = []

        for x in batch:
            batch_mfcc.append(x) # TODO
            lengths_mfcc.append(x.shape[0])

            
        # HINT: CHECK OUT -> pad_sequence (imported above)
        # Also be sure to check the input format (batch_first)
        batch_mfcc_pad = pad_sequence(batch_mfcc, batch_first = True) # TODO

        # You may apply some transformation, Time and Frequency masking, here in the collate function;
        # Food for thought -> Why are we applying the transformation here and not in the __getitem__?
        #                  -> Would we apply transformation on the validation set as well?
        #                  -> Is the order of axes / dimensions as expected for the transform functions?
        
        # Return the following values: padded features, padded labels, actual length of features, actual length of the labels
        return batch_mfcc_pad, torch.tensor(lengths_mfcc)


# Data

In [14]:
pwd = os.getcwd()
train_data = AudioDataset(os.path.join(pwd, "hw3p2", "train-clean-360"), train = True) #train-clean-100
val_data = AudioDataset(os.path.join(pwd,"hw3p2","dev-clean"), train = False)
test_data = AudioDatasetTest(os.path.join(pwd,"hw3p2","test-clean"))

# Cepstral Normalize

In [15]:
import speechpy
import numpy as np

def concat_np(data: list):
    return np.concatenate(data, axis = 0)

def decompose_np(origin_data, target_data):
    idxs = [0]
    for i in range(len(origin_data)):
        length = origin_data.mfccs[i].shape[0]
        idxs.append(idxs[i] + length)
    idxs = idxs[1:-1]
    return np.split(target_data, idxs)
    
def noramlize_cmvn(train_data, val_data, test_data):
    tr, val, te = concat_np(train_data.mfccs), concat_np(val_data.mfccs), concat_np(test_data.mfccs)
    total_data = concat_np([tr, val, te])
    total_data = speechpy.processing.cmvn(total_data, variance_normalization = True)
    
    train_idx = tr.shape[0]
    test_idx = train_idx + val.shape[0]
    
    n_train = total_data[:train_idx, :]
    n_val = total_data[train_idx:test_idx, :]
    n_test = total_data[test_idx:, :]
    
    train_data.mfccs = decompose_np(train_data, n_train)
    val_data.mfccs = decompose_np(val_data, n_val)
    test_data.mfccs = decompose_np(test_data, n_test)
    

In [16]:
noramlize_cmvn(train_data, val_data, test_data)

### Data - Hyperparameters

In [17]:
# BATCH_SIZE = 32 # Increase if your device can handle it

# transforms = [] # set of tranformations
# # You may pass this as a parameter to the dataset class above
# # This will help modularize your implementation

# root = '/content/hw3p2' 

### Data loaders

In [18]:
# get me RAMMM!!!! 
import gc 
gc.collect()

48

In [19]:
# Do NOT forget to pass in the collate function as parameter while creating the dataloader
train_loader = DataLoader(train_data, batch_size = config["batch_size"], 
                          shuffle = True, drop_last = False, collate_fn = train_data.collate_fn,
                           pin_memory = True) #TODO

val_loader = DataLoader(val_data, batch_size = config["batch_size"], 
                        shuffle = False, drop_last = False, collate_fn = val_data.collate_fn,
                         pin_memory = True)

test_loader = DataLoader(test_data, batch_size = config["batch_size"], 
                         shuffle = False, drop_last = False, collate_fn = test_data.collate_fn,
                          pin_memory = True)


In [20]:
print("Batch size: ", config["batch_size"])
print("Train dataset samples = {}, batches = {}".format(train_data.__len__(), len(train_loader)))
print("Val dataset samples = {}, batches = {}".format(val_data.__len__(), len(val_loader)))
print("Test dataset samples = {}, batches = {}".format(test_data.__len__(), len(test_loader)))

Batch size:  32
Train dataset samples = 104014, batches = 3251
Val dataset samples = 2703, batches = 85
Test dataset samples = 2620, batches = 82


In [21]:
# sanity check
for data in train_loader:
    x, y, lx, ly = data
    print(x.shape, y.shape, lx.shape, ly.shape)
    break 

torch.Size([32, 1697, 15]) torch.Size([32, 201]) torch.Size([32]) torch.Size([32])


# Model Config

In [22]:
OUT_SIZE = len(LABELS)
OUT_SIZE

43

## Basic

In [23]:
from torch.autograd import Variable

h_0 = Variable(torch.zeros(2, 3, 512), requires_grad = True)
c_0 = Variable(torch.zeros(2, 3, 512), requires_grad = True)

In [24]:
from torch.autograd import Variable
from collections import Counter

def DS_block(in_channel, out_channel, stride):
    return nn.Sequential(
                         nn.Conv1d(in_channel, in_channel, kernel_size = 3,  stride = stride, groups = in_channel, padding = 2),
                         nn.BatchNorm1d(in_channel),
                         nn.GELU(), #nn.ReLU(),
        
                         nn.Conv1d(in_channel, out_channel, kernel_size = 1, stride = 1),
                         nn.BatchNorm1d(out_channel),
                         nn.GELU() #nn.ReLU(),
                        )

def Res_Block(in_channel, out_channel):
    return nn.Sequential(
                         nn.Conv1d(in_channel, out_channel, kernel_size = 1, stride = 1),
                         nn.BatchNorm1d(out_channel),
                         nn.GELU(), #nn.ReLU(),
        
                         nn.Conv1d(out_channel, in_channel, kernel_size = 1, stride = 1),
                         nn.BatchNorm1d(in_channel),
                         nn.GELU(), #nn.ReLU(),
                         )

def make_DS_Res_Block(dims, strides):
    ds_dims = dims[:-1]
    rs_dims = dims[1:]

    ds_layer = []
    ds_dims = list(zip(ds_dims[:-1], ds_dims[1:]))
    for idx, (in_dim, out_dim) in enumerate(ds_dims):
        ds_layer += [DS_block(in_dim, out_dim, strides[idx])]

    rs_layer = []
    rs_dims = list(zip(rs_dims[:-1], rs_dims[1:]))
    for in_dim, out_dim in rs_dims:
        rs_layer += [Res_Block(in_dim, out_dim)]
    
    return ds_layer, rs_layer

class Network(nn.Module):

    def __init__(self, dims = None):

        super(Network, self).__init__()

        # Adding some sort of embedding layer or feature extractor might help performance.
        if dims is None:
            #dims = [15, 1024, 1024, 1024, 1024, 1024, 512, 512, 512, 512, 512]
            #dims = [15, 1024, 1024, 1024, 512, 512, 512, ]
            #dims = [15, 512, 512, 512, 256, 256, 256, ]
            self.dims = [15, 1024, 1024, 1024, 512, 512, 512]
            self.strides = [2,1,1,2,1,1]
        
        self.lx_scale = 2 ** Counter(self.strides)[2]
        
        ds_layer, rs_layer = make_DS_Res_Block(self.dims, self.strides)
        self.ds_layer = nn.ModuleList(ds_layer)
        self.rs_layer = nn.ModuleList(rs_layer)

        
        # TODO : look up the documentation. You might need to pass some additional parameters.
        #self.lstm1 = nn.LSTM(input_size = 256, hidden_size = 256, num_layers = 2, batch_first = True, dropout = 0.25) 
        #self.lstm2 = nn.LSTM(input_size = 256, hidden_size = 256, num_layers = 2, batch_first = True, dropout = 0.25)
        
        #self.lstm1 = nn.LSTM(input_size = 512, hidden_size = 256, num_layers = 2, batch_first = True, dropout = 0.25) 
        #self.lstm2 = nn.LSTM(input_size = 256, hidden_size = 256, num_layers = 2, batch_first = True, dropout = 0.25)
        
        self.lstm = nn.LSTM(input_size = 512, hidden_size = 256, num_layers = 3, batch_first = True, dropout = 0.25, bidirectional = True)
        
        
        self.classification = nn.Sequential(
            #TODO: Linear layer with in_features from the lstm module above and out_features = OUT_SIZE
            #nn.Linear(256, OUT_SIZE)
            nn.Linear(512, OUT_SIZE)
        )

        #TODO: Apply a log softmax here. Which dimension would apply it on ?

        self.logSoftmax = nn.LogSoftmax(dim = 2)
        
        
        
    def forward(self, x, lx):
        out = x.permute(0,2,1)
        for i in range(len(self.ds_layer)):
            out = self.ds_layer[i](out)
            out = self.rs_layer[i](out) + out
        out = out.permute(0,2,1)
        
        #out = pack_padded_sequence(out, lx.cpu().numpy(), batch_first=True, enforce_sorted=False)
        
#         out, (hn, cn) = self.lstm1(out)
#         out, (hn, cn) = self.lstm2(out)
        out, _ = self.lstm(out)
        
        #out, out_length = pad_packed_sequence(out, batch_first=True)
        
        out = self.classification(out)
        out = self.logSoftmax(out)
        
        return out, lx // self.lx_scale  #out_length
    
    

## INIT

In [25]:
torch.cuda.empty_cache()
model = Network().to(device)
#model = torch.nn.DataParallel(model, device_ids=[0, 1])

summary(model, x.to(device), lx) # x and lx come from the sanity check above :)

                                Kernel Shape     Output Shape    Params  \
Layer                                                                     
0_ds_layer.0.Conv1d_0             [1, 15, 3]    [32, 15, 850]      60.0   
1_ds_layer.0.BatchNorm1d_1              [15]    [32, 15, 850]      30.0   
2_ds_layer.0.GELU_2                        -    [32, 15, 850]         -   
3_ds_layer.0.Conv1d_3          [15, 1024, 1]  [32, 1024, 850]   16.384k   
4_ds_layer.0.BatchNorm1d_4            [1024]  [32, 1024, 850]    2.048k   
5_ds_layer.0.GELU_5                        -  [32, 1024, 850]         -   
6_rs_layer.0.Conv1d_0        [1024, 1024, 1]  [32, 1024, 850]   1.0496M   
7_rs_layer.0.BatchNorm1d_1            [1024]  [32, 1024, 850]    2.048k   
8_rs_layer.0.GELU_2                        -  [32, 1024, 850]         -   
9_rs_layer.0.Conv1d_3        [1024, 1024, 1]  [32, 1024, 850]   1.0496M   
10_rs_layer.0.BatchNorm1d_4           [1024]  [32, 1024, 850]    2.048k   
11_rs_layer.0.GELU_5     

,Kernel Shape,Output Shape,Params,Mult-Adds
Layer,,,,
0_ds_layer.0.Conv1d_0,"[1, 15, 3]","[32, 15, 850]",60.0,38250.0
1_ds_layer.0.BatchNorm1d_1,[15],"[32, 15, 850]",30.0,15.0
2_ds_layer.0.GELU_2,-,"[32, 15, 850]",NaN,NaN
3_ds_layer.0.Conv1d_3,"[15, 1024, 1]","[32, 1024, 850]",16384.0,13056000.0
4_ds_layer.0.BatchNorm1d_4,[1024],"[32, 1024, 850]",2048.0,1024.0
...,...,...,...,...
58_rs_layer.4.BatchNorm1d_4,[512],"[32, 512, 430]",1024.0,512.0
59_rs_layer.4.GELU_5,-,"[32, 512, 430]",NaN,NaN
60_lstm,-,"[32, 430, 512]",4730880.0,4718592.0


In [26]:
out, out_lengths = model(x.to(device), lx)
criterion = nn.CTCLoss()
loss = criterion(out.permute(1,0,2), y, out_lengths, ly)

In [27]:
loss

tensor(6.4160, device='cuda:0', grad_fn=<MeanBackward0>)

# Training Config

In [28]:
train_config = {
    "beam_width" : 2,
    "lr" : 2e-3,
    "epochs" : 50
    } # Feel free to add more items here

In [50]:
#TODO
from torch.optim.lr_scheduler import StepLR 

criterion = nn.CTCLoss()

# Define CTC loss as the criterion. How would the losses be reduced?
# CTC Loss: https://pytorch.org/docs/stable/generated/torch.nn.CTCLoss.html
# Refer to the handout for hints

optimizer =  torch.optim.AdamW(model.parameters(), lr = 0.002) # What goes in here?

# Declare the decoder. Use the CTC Beam Decoder to decode phonemes
# CTC Beam Decoder Doc: https://github.com/parlance/ctcdecode
decoder = CTCBeamDecoder(
                         labels = LABELS,
                         beam_width = 5,
                         num_processes = 40,
                         log_probs_input = True
                        ) #TODO 

scheduler = StepLR(optimizer, step_size = 30, gamma=0.1)#TODO

# Mixed Precision, if you need it
scaler = torch.cuda.amp.GradScaler()

### Levenshtein

In [51]:
# Use debug = True to see debug outputs

from Levenshtein import distance as lev

def calculate_levenshtein(h, y, lh, ly, decoder, labels, debug = False):

    if debug:
        print(f"\n----- IN LEVENSHTEIN -----\n")
        print("h", h.shape)
        print("y", y.shape)
        print("lh", lh.shape)
        print('ly', ly.shape)
        # Add any other debug statements as you may need
        # you may want to use debug in several places in this function
        
        
    # TODO: look at docs for CTC.decoder and find out what is returned here
    beam_results, beam_scores, timesteps, out_seq_len = decoder.decode(h, seq_lens = lh)
    print("beam_scores",beam_scores)
    print("out_seq_len",out_seq_len)

    batch_size = beam_results.shape[0] # TODO
    distance = 0 # Initialize the distance to be 0 initially

    # TODO: Loop through each element in the batch
    targets = []
    for i in range(batch_size):
        target = beam_results[i][0][:out_seq_len[i][0]]
        t_string = "".join([LABELS[idx] for idx in target])
        y_string = "".join([LABELS[idx] for idx in y[i][:ly[i]]])
#         print(target, y[i][:ly[i]])
#         distance += lev(target, y[i][:ly[i]])
        #print(t_string, y_string)
        distance += lev(t_string, y_string)
        
    

    distance /= batch_size # TODO: Uncomment this, but think about why we are doing this

    return distance

In [52]:
calculate_levenshtein(out,y,out_lengths, ly, decoder, LABELS, debug=True)


----- IN LEVENSHTEIN -----

h torch.Size([32, 430, 43])
y torch.Size([32, 201])
lh torch.Size([32])
ly torch.Size([32])
beam_scores tensor([[1046.8346, 1046.8646, 1047.0068, 1047.6099, 1047.6266],
        [1162.8406, 1163.1364, 1163.1370, 1163.1915, 1163.1938],
        [1100.2616, 1100.2634, 1100.5109, 1100.8140, 1100.8335],
        [ 884.6694,  885.5419,  885.5541,  885.5679,  885.5738],
        [1081.1993, 1081.2040, 1082.0872, 1082.0890, 1082.0952],
        [ 257.1790,  257.2653,  257.2874,  257.8599,  257.8665],
        [1292.6403, 1292.6411, 1292.8605, 1292.8793, 1292.8993],
        [1213.2466, 1213.2513, 1214.1342, 1214.1355, 1214.1389],
        [1262.6636, 1262.6708, 1263.0203, 1263.0248, 1263.0250],
        [1051.1140, 1051.1462, 1051.1501, 1051.8245, 1051.8301],
        [1000.6113, 1000.6381, 1000.6389, 1001.1587, 1001.1763],
        [1076.5792, 1076.8888, 1076.8916, 1076.9205, 1076.9214],
        [1145.0446, 1145.0648, 1145.2179, 1145.2297, 1145.8020],
        [1224.0615, 12

289.34375

In [ ]:
# ANOTEHR SANITY CHECK

with torch.no_grad():
    for i, data in enumerate(train_loader):
        optimizer.zero_grad()
        x, y, lx, ly = data
        x = x.to(device)
        y = y.to(device)
        
        out, out_lengths = model(x, lx)
        
        loss = criterion(out.permute(1,0,2), y, out_lengths, ly).double().float()
        
      #TODO: 
      # Follow the following steps, and 
      # Add some print statements here for sanity checking
      
      #1. What values are you returning from the collate function

      #2. Move the features and target to <DEVICE>
    
      #3. Print the shapes of each to get a fair understanding 
      #4. Pass the inputs to the model
            # Think of the following before you implement:
            # 4.1 What will be the input to your model?
            # 4.2 What would the model output?
            # 4.3 Print the shapes of the output to get a fair understanding 

      # Calculate loss: https://pytorch.org/docs/stable/generated/torch.nn.CTCLoss.html
      # Calculating the loss is not straightforward. Check the input format of each parameter
        print(f"loss: {loss}")
        distance = calculate_levenshtein(out, y, out_lengths, ly, decoder, LABELS, debug = False)
        print(f"lev-distance: {distance}")
        
        
        #loss.backward()
        #optimizer.step()

        break # one iteration is enough

# Training

### Eval function
Writing a function to do one round of evaluations will help make your code more modular, you can, however, choose to skip this if you'd like it.

In [ ]:
torch.cuda.empty_cache()
def evaluate(data_loader, model, decoder, LABELS, device):
    
    dist = 0
    total_loss = 0
    batch_bar = tqdm(total=len(data_loader), dynamic_ncols=True, leave=False, position=0, desc='Val') 
    # TODO Fill this function out, if you're using it.
    model.eval()
    for x, y, lx, ly in data_loader:
        x = x.to(device)
        y = y.to(device)
        
        with torch.no_grad():
            out, out_lengths = model(x, lx)
            
        loss = criterion(out.permute(1,0,2), y, out_lengths, ly)
        total_loss += loss.item()
        
        dist += calculate_levenshtein(out, y, out_lengths, ly,
                                     decoder, LABELS)
        
    total_loss /= len(data_loader)
    dist /= len(data_loader)
    return total_loss, dist


In [ ]:
evaluate(val_loader, model, decoder, LABELS, device)

### Training Setup

In [ ]:
# This is for checkpointing, if you're doing it over multiple sessions
# last_epoch_completed = 0
# start = last_epoch_completed
# end = epochs
# dist_freq = 1

Again, writing a train step might help you code be more modular. You may choose to skip this and write the whole thing out in the training loop below if you so wish.

In [ ]:
def train_step(train_loader, model, optimizer, criterion, scheduler, device, scaler = None):
    
    batch_bar = tqdm(total=len(train_loader), dynamic_ncols=True, leave=False, position=0, desc='Train') 
    train_loss = 0
    
    model.train()
    for i, data in enumerate(train_loader):

        x, y, lx, ly = data
        x = x.to(device)
        y = y.to(device)

        # TODO: Fill this with the help of your sanity check
        out, out_lengths = model(x, lx)

        loss = criterion(out.permute(1,0,2), y, out_lengths, ly)

        # HINT: Are you using mixed precision? 

        batch_bar.set_postfix(
            loss = f"{train_loss/ (i+1):.4f}",
            lr = f"{optimizer.param_groups[0]['lr']}"
        )

        train_loss += loss.item()
        batch_bar.update()
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    batch_bar.close()
    train_loss /= len(train_loader) # TODO

    return train_loss # And anything else you may wish to get out of this function

### Train Loop

In [ ]:
config

In [ ]:
torch.cuda.empty_cache()
gc.collect()

#TODO: Please complete the training loop

best_val_dist = float("inf") # if you're restarting from some checkpoint, use what you saw there.
for epoch in range(config["epochs"]):
    print(f"epoch: {epoch}/{config['epochs']}")
    # one training step
    train_loss = train_step(train_loader, model, optimizer, criterion, scheduler, device)

    # one validation step (if you want)
    val_loss, val_dist = evaluate(val_loader, model, decoder, LABELS, device)

    # HINT: Calculating levenshtein distance takes a long time. Do you need to do it every epoch?
    # Does the training step even need it? 

    # Where you have your scheduler.step depends on the scheduler you use.
    scheduler.step()
    
    # Use the below code to save models
    if val_dist < best_val_dist:
        print("Saving model")
        model_path = f"{config['model_save']}/{config['project']}.pth"

        torch.save({'model_state_dict':model.state_dict(),
                  'optimizer_state_dict':optimizer.state_dict(),
                  'val_dist': val_dist, 
                  'epoch': epoch}, model_path)
        best_val_dist = val_dist
        wandb.save(model_path)
    

    # You may want to log some hyperparameters and results on wandb
    curr_lr = float(optimizer.param_groups[0]['lr'])
    wandb.log({"train_loss":train_loss, 'validation_loss': val_loss, "val_dist": val_dist, 
               "learning_Rate": curr_lr})

run.finish()

In [ ]:
dic = torch.load(model_path)

In [ ]:
model.load_state_dict(dic["model_state_dict"])

# Generate Predictions and Submit to Kaggle

In [ ]:
#TODO: Make predictions

# Follow the steps below:
# 1. Create a new object for CTCBeamDecoder with larger (why?) number of beams
# 2. Get prediction string by decoding the results of the beam decoder

decoder_test =  CTCBeamDecoder(
                         labels = LABELS,
                         beam_width = 20,
                         num_processes = 40,
                         log_probs_input = True
                        ) 

def make_output(h, lh, decoder, LABELS):
  
    beam_results, beam_scores, timesteps, out_seq_len = decoder_test.decode(h, seq_lens = lh) #TODO: What parameters would the decode function take in?
    batch_size = beam_results.shape[0] #What is the batch size

    dist = 0
    preds = []
    for i in range(batch_size): # Loop through each element in the batch

        h_sliced = beam_results[i][0][:out_seq_len[i][0]] #TODO: Obtain the beam results
        h_string = "".join([LABELS[idx] for idx in h_sliced])#TODO: Convert the beam results to phonemes
        preds.append(h_string)
    
    return preds

In [ ]:
#TODO:
# Write a function (predict) to generate predictions and submit the file to Kaggle
def predict(test_loader, model, decoder, LABELS, device):
    
    model.eval()
    preds = []
    with torch.no_grad():
        for x, lx in test_loader:
            x = x.to(device)
            out, out_lengths = model(x, lx)
            preds += make_output(out, out_lengths, decoder, LABELS)

    return preds

torch.cuda.empty_cache()
predictions = predict(test_loader, model, decoder_test, LABELS, device)


In [ ]:
import pandas as pd

path = os.path.join(os.getcwd(),'hw3p2/test-clean/transcript/random_submission.csv')

df = pd.read_csv(path)
df.label = predictions

df.head()

In [ ]:
df.to_csv(f'{config["project"]}.csv', index = False)

In [ ]:
#!kaggle competitions submit -c 11-785-f22-hw3p2 -f f'{config["project"]}.csv' -m "Message"

In [ ]:
#!kaggle competitions submit -c 11-785-f22-hw3p2 -f f'{config["project"]}.csv' -m "Message"
command = f"kaggle competitions submit -c 11-785-f22-hw3p2 -f {config['project']}.csv -m Message"
command

# 시도해보기

<br> 1) Data Prallel -> Batch_True = False
<br> 2) DownSampling -> CNN stride를 2이상으로 바꾸기 및 굳이 packed seq 쓰지 않기. -> 이때 output_lengts 대신 lx를 쓰면 된당
<br>

In [ ]:
# !kaggle competitions submit -c 11-785-f22-hw3p2 -f LSTM_3_layers.csv -m Message


In [ ]:
# !kaggle competitions submit -c 11-785-f22-hw3p2 -f LSTM_3_layers_epoch31.csv -m Message
